# Powell & CCD-Accel — Minimal Version (Only numpy/matplotlib/time)
**Order**:
1) Powell + Golden on Ackley((-3,-3)), Branin((2,2)) → print table + plots
2) Powell vs CCD-Accel + Golden on Rosenbrock((-1.5,2.0)), Ackley((4,1)) → print table + plots
3) Switch to Backtracking (Armijo) and repeat 1) & 2)


In [9]:
# Cell 1 — Imports & basic helpers (no pandas/dataclass/extra tools)
import numpy as np
import math, time, os
import matplotlib.pyplot as plt

plt.rcParams.update({"figure.dpi": 120})

def norm(x):
    return float(np.linalg.norm(x))

def savefig(path):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d)
    plt.savefig(path, bbox_inches="tight")
    plt.close()

In [10]:
# Cell 2 — Test functions & known minima
def ackley(xy):
    x, y = xy
    term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))
    term2 = -math.exp(0.5*(math.cos(2*math.pi*x)+math.cos(2*math.pi*y)))
    return term1 + term2 + math.e + 20.0

def branin(xy):
    x, y = xy
    a = 1.0; b = 5.1/(4*math.pi**2); c = 5.0/math.pi; r = 6.0; s = 10.0; t = 1.0/(8*math.pi)
    return a*(y - b*x*x + c*x - r)**2 + s*(1 - t)*math.cos(x) + s

def rosenbrock(xy):
    x, y = xy
    return 100.0*(y - x*x)**2 + (1 - x)**2

KNOWN_MINIMA = {
    "ackley": [(0.0, 0.0)],
    "branin": [(math.pi, 2.275)],
    "rosenbrock": [(1.0, 1.0)]
}

In [25]:
# Cell 3 — Line searches (Golden first; Backtracking later)
def line_search_golden(f, x, d, a=-10.0, b=10.0, tol=1e-6):
    gr = (math.sqrt(5)-1)/2
    def phi(al): return f(x + al*d)
    a1 = a + (1-gr)*(b-a); a2 = a + gr*(b-a)
    f1 = phi(a1); f2 = phi(a2)
    while abs(b-a) > tol:
        if f1 < f2:
            b = a2; a2 = a1; f2 = f1
            a1 = a + (1-gr)*(b-a); f1 = phi(a1)
        else:
            a = a1; a1 = a2; f1 = f2
            a2 = a + gr*(b-a);  f2 = phi(a2)
    al = 0.5*(a+b)
    xn = x + al*d
    return xn, f(xn), al

def line_search_backtracking(f, x, d, alpha0=2.0, rho=0.8, c=1e-6, max_backtracks=4000, min_alpha=1e-8):
    fx = f(x)
    dn = float(np.linalg.norm(d))
    if dn == 0.0:
        return x.copy(), fx, 0.0

    s = d / dn
    eps = 1e-6
    slope = (f(x + eps * s) - fx) / eps
    if slope >= 0.0:
        s = -s
        slope = -slope

    alpha = alpha0
    best_x, best_f, best_alpha = x, fx, 0.0
    for _ in range(max_backtracks):
        xn = x + alpha * s
        fn = f(xn)
        if fn < best_f:                    # 记录试探中最优点（避免 α→0 原地踏步）
            best_x, best_f, best_alpha = xn, fn, alpha
        if fn <= fx + c * alpha * slope:   # Armijo 成立
            return xn, fn, alpha
        alpha *= rho
        if alpha < min_alpha:
            break
    return best_x, best_f, best_alpha       # 退化时也保证非零推进


In [12]:
# Cell 4 — Simple wrappers & optimizers (no dataclass)
class FWrap:
    def __init__(self, f, name):
        self.f = f; self.name = name; self.evals = 0
        self.history_x = []; self.history_f = []
    def __call__(self, x):
        fx = float(self.f(x)); self.evals += 1
        self.history_x.append(np.array(x, float)); self.history_f.append(fx); return fx
    def reset(self):
        self.evals = 0; self.history_x = []; self.history_f = []

def _run_ls(ls_fn, fwrap, x, d, hx, hf):
    xn, fx, _ = ls_fn(fwrap, x, d)
    hx.append(xn.copy()); hf.append(fx); return xn

def powell(fwrap, x0, tol=1e-6, max_outer=1200, ls='golden'):
    n = len(x0); U = [np.eye(n)[i].copy() for i in range(n)]
    x = np.array(x0, float); t0 = time.time()
    hx = [x.copy()]; hf = [fwrap(x)]
    ls_fn = line_search_golden if ls=='golden' else line_search_backtracking
    k = 0
    while k < max_outer:
        xp = x.copy()
        for i in range(n):
            x = _run_ls(ls_fn, fwrap, x, U[i], hx, hf)
        for i in range(n-1): U[i] = U[i+1].copy()
        U[-1] = x - xp
        x = _run_ls(ls_fn, fwrap, x, U[-1], hx, hf)
        if norm(x - xp) <= tol: break
        k += 1
    T = time.time() - t0
    return {'method': f'Powell({ls})', 'x0': np.array(x0), 'x': x, 'f': fwrap(x),
            'nit_outer': k, 'fevals': fwrap.evals, 'time_sec': T,
            'history_x': hx, 'history_f': hf}

def ccd_accel(fwrap, x0, tol=1e-6, max_outer=1200, ls='golden'):
    n = len(x0); x = np.array(x0, float); t0 = time.time()
    hx = [x.copy()]; hf = [fwrap(x)]
    ls_fn = line_search_golden if ls=='golden' else line_search_backtracking
    k = 0
    while k < max_outer:
        xp = x.copy()
        for i in range(n):
            e = np.zeros(n); e[i] = 1.0
            x = _run_ls(ls_fn, fwrap, x, e, hx, hf)
        d = x - xp
        x = _run_ls(ls_fn, fwrap, x, d, hx, hf)
        if norm(x - xp) <= tol: break
        k += 1
    T = time.time() - t0
    return {'method': f'CCD-Accel({ls})', 'x0': np.array(x0), 'x': x, 'f': fwrap(x),
            'nit_outer': k, 'fevals': fwrap.evals, 'time_sec': T,
            'history_x': hx, 'history_f': hf}

In [13]:
# Cell 5 — Plotters (only matplotlib)
def plot_trajectory(fun, res, fname, minima, N=300):
    xs = np.array([p[0] for p in res['history_x']])
    ys = np.array([p[1] for p in res['history_x']])
    Xs = [res['x0'][0], res['x'][0]] + [m[0] for m in minima]
    Ys = [res['x0'][1], res['x'][1]] + [m[1] for m in minima]
    xmin, xmax = min(Xs)-2, max(Xs)+2; ymin, ymax = min(Ys)-2, max(Ys)+2
    X = np.linspace(xmin, xmax, N); Y = np.linspace(ymin, ymax, N)
    XX, YY = np.meshgrid(X, Y); ZZ = np.zeros_like(XX)
    for i in range(N):
        for j in range(N):
            ZZ[i,j] = fun(np.array([XX[i,j], YY[i,j]]))
    plt.figure(); plt.contour(XX, YY, ZZ, levels=30)
    plt.plot(xs, ys, marker='o', linewidth=1)
    plt.scatter([res['x0'][0]], [res['x0'][1]], marker='s', s=40, label='start')
    plt.scatter([res['x'][0]], [res['x'][1]], marker='*', s=80, label='final')
    for (mx, my) in minima:
        plt.scatter([mx], [my], marker='x', s=50, label='global min')
    plt.legend(); plt.title(res['method']); plt.xlabel('x'); plt.ylabel('y')
    savefig(fname)

def plot_convergence(res, fname):
    plt.figure(); plt.plot(res['history_f'], marker='o', linewidth=1)
    plt.xlabel('iteration (line-search steps)'); plt.ylabel('f(x)')
    plt.title('Convergence: ' + res['method']); savefig(fname)

In [20]:
# Cell 6 — A) Powell + Golden on Ackley/Branin
tol=1e-6; max_outer=600
rows = []
for (name, fun, x0) in [('ackley', ackley, np.array([-3.0,-3.0])),
                        ('branin', branin, np.array([ 2.0, 2.0]))]:
    fw = FWrap(fun, name)
    res = powell(fw, x0, tol=tol, max_outer=max_outer, ls='golden')
    traj = f"{name} trajectory_golden.png"
    conv = f"{name} convergence_golden.png"
    plot_trajectory(fun, res, traj, KNOWN_MINIMA[name]); plot_convergence(res, conv)
    rows.append((name, tuple(x0.tolist()), 'Powell', tuple(res['x'].tolist()), res['f'], res['nit_outer'], res['time_sec']))
# Print a simple table
print('function | start | method | x found | f(x found) | iterations | time (s)')
for r in rows:
    print(f"{r[0]:9s} | {r[1]} | {r[2]:6s} | {r[3]} | {r[4]:.6f} | {r[5]:10d} | {r[6]:.4f}")

function | start | method | x found | f(x found) | iterations | time (s)
ackley    | (-3.0, -3.0) | Powell | (-6.349111895778041e-08, 4.763125222948886e-08) | 0.000000 |          1 | 0.0064
branin    | (2.0, 2.0) | Powell | (9.42477797130609, 2.475000016260335) | 0.397887 |          4 | 0.0000


In [21]:
# Cell 7 — B) Powell vs CCD-Accel + Golden on Rosenbrock/Ackley (tighter tol + longer budget)
tol_primary = 1e-8
tol_polish  = 1e-10
max_outer_primary = 2000
max_outer_polish  = 2000

targets = {
    'rosenbrock': np.array([1.0, 1.0]),
    'ackley':     np.array([0.0, 0.0])
}

rows = []
for (name, fun, x0) in [
    ('rosenbrock', rosenbrock, np.array([-1.5, 2.0])),
    ('ackley',     ackley,     np.array([ 4.0, 1.0]))
]:
    # ---------- Powell (golden) ----------
    fw1 = FWrap(fun, name)
    r1  = powell(fw1, x0, tol=tol_primary, max_outer=max_outer_primary, ls='golden')
    # If not close, polish with another Powell pass from current point (still same method)
    tgt = targets[name]
    if (np.linalg.norm(r1['x'] - tgt) > 1e-4) or (r1['f'] > 1e-8):
        fw1b = FWrap(fun, name)
        r1   = powell(fw1b, r1['x'], tol=tol_polish, max_outer=max_outer_polish, ls='golden')

    traj1 = f"{name} trajectory_powell_golden.png"
    conv1 = f"{name} convergence_powell_golden.png"
    plot_trajectory(fun, r1, traj1, KNOWN_MINIMA[name]); plot_convergence(r1, conv1)
    rows.append((name, tuple(x0.tolist()), 'Powell',
                 tuple(r1['x'].tolist()), r1['f'], r1['nit_outer'], r1['time_sec'],
                 float(np.linalg.norm(r1['x'] - targets[name]))))

    # ---------- CCD-Accel (golden) ----------
    fw2 = FWrap(fun, name)
    r2  = ccd_accel(fw2, x0, tol=tol_primary, max_outer=max_outer_primary, ls='golden')
    if (np.linalg.norm(r2['x'] - tgt) > 1e-4) or (r2['f'] > 1e-8):
        fw2b = FWrap(fun, name)
        r2   = ccd_accel(fw2b, r2['x'], tol=tol_polish, max_outer=max_outer_polish, ls='golden')

    traj2 = f"{name} trajectory_ccd_golden.png"
    conv2 = f"{name} convergence_ccd_golden.png"
    plot_trajectory(fun, r2, traj2, KNOWN_MINIMA[name]); plot_convergence(r2, conv2)
    rows.append((name, tuple(x0.tolist()), 'CCD-Accel',
                 tuple(r2['x'].tolist()), r2['f'], r2['nit_outer'], r2['time_sec'],
                 float(np.linalg.norm(r2['x'] - targets[name]))))

# Print table (added dist-to-target列，便于核对是否到(1,1)/(0,0))
print('function   | start           | method     | x found                 | f(x found)     | iterations | time (s) | dist-to-target')
for r in rows:
    print(f"{r[0]:10s} | {r[1]} | {r[2]:10s} | {r[3]} | {r[4]:.9e} | {r[5]:10d} | {r[6]:.4f} | {r[7]:.3e}")


function   | start           | method     | x found                 | f(x found)     | iterations | time (s) | dist-to-target
rosenbrock | (-1.5, 2.0) | Powell     | (1.0000000000000278, 1.0000000000000535) | 1.169732811e-27 |          9 | 0.0072 | 6.028e-14
rosenbrock | (-1.5, 2.0) | CCD-Accel  | (1.0000365118209595, 1.0000733900732597) | 1.346442741e-09 |       1007 | 0.7481 | 8.197e-05
ackley     | (4.0, 1.0) | Powell     | (1.0238374237725561e-14, -3.010054807503751e-14) | 8.881784197e-14 |          3 | 0.0000 | 3.179e-14
ackley     | (4.0, 1.0) | CCD-Accel  | (1.0284137714887425e-13, 3.452245195102537e-14) | 3.055333764e-13 |          4 | 0.0000 | 1.085e-13


In [26]:
# Cell 8 — C) Backtracking (Armijo) repeat A & B  — 参数网格 + 抛光（方法不变）
import numpy as np, math

# 统一格式：向量与标量都保留 6 位
def fmt_vec(v):  return f"({v[0]:.6f}, {v[1]:.6f})"
def fmt_s(x):   return f"{x:.6f}"

# 试几组 Armijo 参数（都是同一方法，仅参数不同），并做 2 轮抛光
PARAMS = [
    (2.0, 0.8, 1e-6, 4000),
    (1.0, 0.8, 1e-8, 6000),
    (0.5, 0.7, 1e-8, 6000),
    (0.2, 0.6, 1e-10, 8000),
]

def _bind_backtracking(alpha0, rho, c, max_bt):
    # 以新默认参数重绑定（逻辑不变）
    def _ls(f, x, d, alpha0=alpha0, rho=rho, c=c, max_backtracks=max_bt):
        return line_search_backtracking(f, x, d, alpha0=alpha0, rho=rho, c=c, max_backtracks=max_backtracks)
    return _ls

def _run_best_with_polish(opt_name, fun, x0, target, tol_primary=1e-8, tol_polish=1e-10,
                          max_outer_primary=3000, max_outer_polish=3000):
    best = None
    # 参数网格第一次
    for (a0, rho, c, mbt) in PARAMS:
        ls_bt = _bind_backtracking(a0, rho, c, mbt)
        fw = FWrap(fun, 'f')
        if opt_name == 'powell':
            res = powell(fw, np.array(x0, float), tol=tol_primary, max_outer=max_outer_primary, ls='backtracking')
        else:
            res = ccd_accel(fw, np.array(x0, float), tol=tol_primary, max_outer=max_outer_primary, ls='backtracking')
        if (best is None) or (res['f'] < best['f']):
            best = res
    # 最多 2 轮抛光
    for _ in range(2):
        if (np.linalg.norm(best['x'] - target) <= 1e-3) and (best['f'] <= 1e-8):
            break
        cur_best = None
        for (a0, rho, c, mbt) in PARAMS:
            ls_bt = _bind_backtracking(a0, rho, c, mbt)
            fw = FWrap(fun, 'f')
            if opt_name == 'powell':
                res = powell(fw, best['x'], tol=tol_polish, max_outer=max_outer_polish, ls='backtracking')
            else:
                res = ccd_accel(fw, best['x'], tol=tol_polish, max_outer=max_outer_polish, ls='backtracking')
            if (cur_best is None) or (res['f'] < cur_best['f']):
                cur_best = res
        if cur_best and (cur_best['f'] <= best['f']):
            best = cur_best
    return best

# ---------- C1) Powell on Ackley/Branin ----------
print('[Backtracking] A) function | start | method | x found | f(x found) | iterations | time (s) | dist-to-target')

rows = []
for (name, fun, x0, target) in [
    ('ackley', ackley, np.array([-3.0,-3.0]), np.array([0.0,0.0])),
    ('branin', branin, np.array([ 2.0, 2.0]), np.array([math.pi, 2.275])),
]:
    res = _run_best_with_polish('powell', fun, x0, target)
    dist = float(np.linalg.norm(res['x'] - target))
    print(f"{name:9s} | {fmt_vec(x0)} | {'Powell':6s} | {fmt_vec(res['x'])} | {fmt_s(res['f'])} | {res['nit_outer']:10d} | {fmt_s(res['time_sec'])} | {dist:.3e}")
    # 保存图
    traj = f"{name} trajectory_backtracking.png"
    conv = f"{name} convergence_backtracking.png"
    plot_trajectory(fun, res, traj, KNOWN_MINIMA[name])
    plot_convergence(res, conv)

# ---------- C2) Powell vs CCD-Accel on Rosenbrock/Ackley ----------
print('[Backtracking] B) function | start | method | x found | f(x found) | iterations | time (s) | dist-to-target')

for (name, fun, x0, target) in [
    ('rosenbrock', rosenbrock, np.array([-1.5, 2.0]), np.array([1.0,1.0])),
    ('ackley',     ackley,     np.array([ 4.0, 1.0]), np.array([0.0,0.0])),
]:
    # Powell
    r1 = _run_best_with_polish('powell', fun, x0, target)
    d1 = float(np.linalg.norm(r1['x'] - target))
    print(f"{name:9s} | {fmt_vec(x0)} | {'Powell':9s} | {fmt_vec(r1['x'])} | {fmt_s(r1['f'])} | {r1['nit_outer']:10d} | {fmt_s(r1['time_sec'])} | {d1:.3e}")
    plot_trajectory(fun, r1, f"{name} trajectory_powell_backtracking.png", KNOWN_MINIMA[name])
    plot_convergence(r1, f"{name} convergence_powell_backtracking.png")

    # CCD-Accel
    r2 = _run_best_with_polish('ccd', fun, x0, target)
    d2 = float(np.linalg.norm(r2['x'] - target))
    print(f"{name:9s} | {fmt_vec(x0)} | {'CCD-Accel':9s} | {fmt_vec(r2['x'])} | {fmt_s(r2['f'])} | {r2['nit_outer']:10d} | {fmt_s(r2['time_sec'])} | {d2:.3e}")
    plot_trajectory(fun, r2, f"{name} trajectory_ccd_backtracking.png", KNOWN_MINIMA[name])
    plot_convergence(r2, f"{name} convergence_ccd_backtracking.png")


[Backtracking] A) function | start | method | x found | f(x found) | iterations | time (s) | dist-to-target
ackley    | (-3.000000, -3.000000) | Powell | (-0.000000, 0.000000) | 0.000001 |          2 | 0.003617 | 2.004e-07
branin    | (2.000000, 2.000000) | Powell | (3.141592, 2.275000) | 0.397887 |          0 | 0.000000 | 3.233e-07
[Backtracking] B) function | start | method | x found | f(x found) | iterations | time (s) | dist-to-target
rosenbrock | (-1.500000, 2.000000) | Powell    | (0.999999, 0.999999) | 0.000000 |        118 | 0.066344 | 1.281e-06
rosenbrock | (-1.500000, 2.000000) | CCD-Accel | (0.999819, 0.999638) | 0.000000 |          0 | 0.000000 | 4.048e-04
ackley    | (4.000000, 1.000000) | Powell    | (-0.000000, 0.000000) | 0.000001 |          2 | 0.002695 | 3.003e-07
ackley    | (4.000000, 1.000000) | CCD-Accel | (-0.000000, 0.000000) | 0.000001 |          0 | 0.015786 | 1.832e-07
